# Feature Engineering for Classification

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import datetime
import re
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
%matplotlib inline

In [3]:
za_data = pd.read_csv("Classification/all_files_globalv1.csv")

In [4]:
za_data.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'artist_names',
       'track_name', 'rank', 'uri', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [5]:
len(list(za_data.columns))

25

In [6]:
za_data["End Date Dt"] = za_data["End Date Dt"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [7]:
za_data.shape

(87582, 25)

In [8]:
za_data_test = za_data[za_data['End Date Dt'] > datetime(2023, 5, 31)]

za_data_test_first_time = za_data_test[za_data_test["trackAppearanceCount"] == 0]
za_data_test_first_time_df = za_data_test_first_time[['main_artist', 'track_name', "trackAppearanceCount", "artistAppearanceCount", "rank", "peak_rank", 'previous_rank', 'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference']]


za_data_test = za_data_test[za_data_test["trackAppearanceCount"] != 0]
za_data = za_data[za_data['End Date Dt'] <= datetime(2023, 5, 31)]

In [9]:
za_data_df = za_data[['main_artist', 'track_name', "trackAppearanceCount", "artistAppearanceCount", "rank", "peak_rank", 'previous_rank', 'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference']]
za_data_test_df = za_data_test[['main_artist', 'track_name', "trackAppearanceCount", "artistAppearanceCount", "rank", "peak_rank", 'previous_rank', 'weeks_on_chart', 'streams', 'ArtistCount', 'isTopTen', 'rank difference']]

In [10]:
# za_data["End Date"].unique().tolist()

In [11]:
za_data_test_first_time_df

,main_artist,track_name,trackAppearanceCount,artistAppearanceCount,rank,peak_rank,previous_rank,weeks_on_chart,streams,ArtistCount,isTopTen,rank difference
0,$,BACK TO ME,0,0,24,24,-1,1,19028110,1,0,0
2,$,BEG FORGIVENESS,0,11,89,89,-1,1,11802992,1,0,0
3,$,BURN,0,12,21,21,-1,1,19646355,1,0,0
10,$,CARNIVAL,0,1,4,4,-1,1,32277135,1,1,0
30,$,DO IT,0,13,59,59,-1,1,14290306,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
87443,juan karlos,ERE,0,0,101,101,-1,1,10257602,1,0,0
87480,sombr,back to friends,0,0,83,83,-1,1,11666265,1,1,0
87488,sombr,undressed,0,2,123,123,-1,1,10016074,1,1,0
87510,wifiskeleton,Nope your too late i already died,0,0,111,111,-1,1,10352667,1,0,0


In [12]:
groupby_cols = ['main_artist', 'track_name']

In [13]:
za_data_test_first_time

,trackAppearanceCount,artistAppearanceCount,artist_names,track_name,rank,uri,source,peak_rank,previous_rank,weeks_on_chart,...,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen,Artist and Track,rank difference,Position over Time,IsGirlGroup,IsBoyGroup
0,0,0,"$, Kanye West, Ty Dolla $ign",BACK TO ME,24,spotify:track:1icgLGTpX2fQXKRe4D7w2b,Ye,24,-1,1,...,1.707955e+09,2024-02-01,1,$,0,"$, Kanye West, Ty Dolla $ign; BACK TO ME",0,0,False,False
2,0,11,"$, Kanye West, Ty Dolla $ign",BEG FORGIVENESS,89,spotify:track:4ihV1wv7QmjbkLHaT9lA4s,Ye,89,-1,1,...,1.707955e+09,2024-02-01,1,$,0,"$, Kanye West, Ty Dolla $ign; BEG FORGIVENESS",0,0,False,False
3,0,12,"$, Kanye West, Ty Dolla $ign",BURN,21,spotify:track:04CyMEHliadfQWMUJb1w99,Ye,21,-1,1,...,1.707955e+09,2024-02-01,1,$,0,"$, Kanye West, Ty Dolla $ign; BURN",0,0,False,False
10,0,1,"$, Kanye West, Ty Dolla $ign, Rich The Kid, Pl...",CARNIVAL,4,spotify:track:3w0w2T288dec0mgeZZqoNN,Ye,4,-1,1,...,1.707955e+09,2024-02-01,1,$,1,"$, Kanye West, Ty Dolla $ign, Rich The Kid, Pl...",0,0,False,False
30,0,13,"$, Kanye West, Ty Dolla $ign",DO IT,59,spotify:track:2iGvsJuc2mC4mDVOVMNAP6,Ye,59,-1,1,...,1.707955e+09,2024-02-01,1,$,0,"$, Kanye West, Ty Dolla $ign; DO IT",0,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87443,0,0,juan karlos,ERE,101,spotify:track:2BLdelIp46CHvKkyWeqZHL,Universal Music Philippines Inc.,101,-1,1,...,1.697069e+09,2023-10-01,1,juan karlos,0,juan karlos; ERE,0,0,False,False
87480,0,0,sombr,back to friends,83,spotify:track:0FTmksd2dxiE5e3rWyJXs6,Warner Records,83,-1,1,...,1.743034e+09,2025-03-01,1,sombr,1,sombr; back to friends,0,0,False,False
87488,0,2,sombr,undressed,123,spotify:track:4AajxCEwGEsmHmT4H1TwjY,Warner Records,123,-1,1,...,1.743638e+09,2025-04-01,1,sombr,1,sombr; undressed,0,0,False,False
87510,0,0,"wifiskeleton, i wanna be a jack-o-lantern",Nope your too late i already died,111,spotify:track:0hta2Lb2zKJ7kEnAEZEE3G,9th St. / Hallwood,111,-1,1,...,1.747267e+09,2025-05-01,1,wifiskeleton,0,"wifiskeleton, i wanna be a jack-o-lantern; Nop...",0,0,False,False


### 0. Entry Position into Chart

In [15]:
def findingtheentrypos(za_data):
    
    groupednum_songs = za_data.groupby(groupby_cols)["End Date"].agg("min")
    groupednum_songs_df = pd.DataFrame(groupednum_songs).reset_index()
    groupednum_songs_df.columns = groupby_cols + ["Artist Song Start Date"]
    
    groupednum_songs_df = groupednum_songs_df.drop_duplicates()
    groupednum_songs_df.shape
    
    groupednum_songs_ls = groupednum_songs_df.values.tolist()
    
    thefirstones = []
    for artsongls in groupednum_songs_ls:
        theartist = artsongls[0]
        thetrack = artsongls[1]
        thestartdate = artsongls[2]
        thefirstones.append(za_data[(za_data["main_artist"] == theartist) & (za_data["track_name"] == thetrack) & (za_data["End Date"] == thestartdate)])
        
    first_track_position_df = pd.concat(thefirstones)
    
    first_track_position_df_relevant = first_track_position_df[["main_artist", "track_name", "rank", "End Date"]]
    first_track_position_df_relevant.columns = ["main_artist", "track_name", "entry rank", "End Date"]
    
    groupednum_songs = za_data.groupby(groupby_cols)["End Date"].agg("count")
    groupednum_songs_df = pd.DataFrame(groupednum_songs).reset_index()
    groupednum_songs_df.columns = groupby_cols + ["Artist Song Count"]
    
    return groupednum_songs_df, first_track_position_df_relevant

### 1. Streams

In [17]:
def findtheMinMaxMeanStreams(za_data):
    groupedmax_streams = za_data.groupby(groupby_cols)["streams"].agg("max")
    groupedmax_streams_df = pd.DataFrame(groupedmax_streams).reset_index()
    groupedmax_streams_df.columns = groupby_cols + ["Max Streams per Song"]
    
    groupedmin_streams = za_data.groupby(groupby_cols)["streams"].agg("min")
    groupedmin_streams_df = pd.DataFrame(groupedmin_streams).reset_index()
    groupedmin_streams_df.columns = groupby_cols + ["Min Streams per Song"]
    
    groupedsum_streams = za_data.groupby(groupby_cols)["streams"].agg("sum")
    groupedsum_streams_df = pd.DataFrame(groupedsum_streams).reset_index()
    groupedsum_streams_df.columns = groupby_cols + ["Total Streams per Song"]

    groupedmean_streams = za_data.groupby(groupby_cols)["streams"].agg("mean")
    groupedmean_streams_df = pd.DataFrame(groupedmean_streams).reset_index()
    groupedmean_streams_df.columns = groupby_cols + ["Mean Streams per Song"]
    
    return groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df

### 2. Positions

In [19]:
def findtheMinMaxPositions(za_data):
    groupedpos_streams = za_data.groupby(groupby_cols)["rank"].agg("min")
    groupedpos_streams_df = pd.DataFrame(groupedpos_streams).reset_index()
    groupedpos_streams_df.columns = groupby_cols + ["Min Position per Song"]

    groupedposmax_streams = za_data.groupby(groupby_cols)["rank"].agg("max")
    groupedposmax_streams_df = pd.DataFrame(groupedposmax_streams).reset_index()
    groupedposmax_streams_df.columns = groupby_cols + ["Max Position per Song"]
    
    return groupedpos_streams_df, groupedposmax_streams_df

### Number of songs per artist

In [21]:
def findtheMaxArtistAppearanceTimes(za_data):
    
    groupedpos_streams = za_data.groupby(["main_artist"])["artistAppearanceCount"].agg("max")
    groupedmin_songs_df = pd.DataFrame(groupedpos_streams).reset_index()
    groupedmin_songs_df.columns = ["main_artist"] + ["Max Artist Appearance"]
    
    return groupedmin_songs_df

#### Maximum number of times an artist has appeared on the chart

In [23]:
def findtheMinMaxNumofSongs(za_data):
    
    groupedpos_streams = za_data.groupby(["main_artist"])["Artist Song Count"].agg("max")
    groupedmin_songs_df = pd.DataFrame(groupedpos_streams).reset_index()
    groupedmin_songs_df.columns = ["main_artist"] + ["Max Song Count per artist"]
    
    return groupedmin_songs_df

#### Number of weeks a song spent on the chart per artist

In [25]:
def findtheWeeksonChartperSong(za_data):
    
    groupedpos_streams = za_data.groupby(groupby_cols)["weeks_on_chart"].agg("max")
    groupedmin_songs_df = pd.DataFrame(groupedpos_streams).reset_index()
    groupedmin_songs_df.columns = groupby_cols + ["Max Weeks on Chart per song"]
    
    return groupedmin_songs_df

#### First week streams and first week positions

In [27]:
def findthePosonWeeksonChartperSong(za_data):
    
    za_data_first_time = za_data[za_data["trackAppearanceCount"] == 0]
    za_data_first_time_df = za_data_first_time[['main_artist', 'track_name', "rank", 'streams']]
    za_data_first_time_df.columns = ['main_artist', 'track_name', "first_week_rank", 'first_week_streams']
    
    return za_data_first_time_df

### 3. Dates

## Combine the dfs

In [30]:
def combinedataframes(groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df, groupedpos_streams_df, groupedposmax_streams_df, groupednum_songs_df):
    groupeddf = groupedmax_streams_df
    
    groupeddf["Min Streams per Song"] = groupedmin_streams_df["Min Streams per Song"]
    groupeddf["Total Streams per Song"] = groupedsum_streams_df["Total Streams per Song"]
    groupeddf["Mean Streams per Song"]= groupedmean_streams_df["Mean Streams per Song"]
    groupeddf["Min Position per Song"] = groupedpos_streams_df["Min Position per Song"]
    groupeddf["Max Position per Song"] = groupedposmax_streams_df["Max Position per Song"]
    groupeddf["Artist Song Count"] = groupednum_songs_df["Artist Song Count"]
    
    groupeddf["MaxMin Difference Position"] = groupeddf["Max Position per Song"] - groupeddf["Min Position per Song"]
    groupeddf["MaxMin Difference Streams"] = groupeddf["Max Streams per Song"] - groupeddf["Min Streams per Song"]
    
    groupeddf["MaxMin Ratio Position"] = groupeddf["Max Position per Song"] / groupeddf["Min Position per Song"]
    groupeddf["MaxMin Ratio Streams"] = groupeddf["Max Streams per Song"] / groupeddf["Min Streams per Song"]
    
    groupeddf.drop(["Max Position per Song"], inplace=True, axis=1)
    
    groupeddf_startpos = groupeddf.merge(first_track_position_df_relevant[["main_artist", "track_name", "entry rank"]], left_on=["main_artist", "track_name"], right_on=["main_artist", "track_name"], how="inner")
    
    groupeddf_startpos["main_art_track"] = groupeddf_startpos["main_artist"] + groupeddf_startpos["track_name"]
    
    groupeddf_startpos_regress = groupeddf_startpos[['main_artist', "main_art_track", 'entry rank', 'Min Position per Song']].drop_duplicates()
    
    return groupeddf, groupeddf_startpos_regress
    

In [31]:
# some_artists_of_interest = ["6LACK", "9umba", "BNXN fka Buju", "Black Coffee", "Big Zulu"]
groupednum_songs_df, first_track_position_df_relevant = findingtheentrypos(za_data)
groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df = findtheMinMaxMeanStreams(za_data)
groupedpos_streams_df, groupedposmax_streams_df = findtheMinMaxPositions(za_data)
groupeddf, groupeddf_startpos_regress = combinedataframes(groupedmax_streams_df, groupedmin_streams_df, groupedsum_streams_df, groupedmean_streams_df, groupedpos_streams_df, groupedposmax_streams_df, groupednum_songs_df)

In [32]:
groupednum_songs_dftest, first_track_position_df_relevanttest = findingtheentrypos(za_data_test_first_time)
groupedmax_streams_dftest, groupedmin_streams_dftest, groupedsum_streams_dftest, groupedmean_streams_dftest = findtheMinMaxMeanStreams(za_data_test_first_time)
groupedpos_streams_dftest, groupedposmax_streams_dftest = findtheMinMaxPositions(za_data_test_first_time)
groupeddftest, groupeddf_startpos_regresstest = combinedataframes(groupedmax_streams_dftest, groupedmin_streams_dftest, groupedsum_streams_dftest, groupedmean_streams_dftest, groupedpos_streams_dftest, groupedposmax_streams_dftest, groupednum_songs_dftest)

In [33]:
groupby_cols

['main_artist', 'track_name']

In [34]:
maxsongperartistdf = findtheMinMaxNumofSongs(groupeddf)
weeksonchartpersong = findtheWeeksonChartperSong(za_data)
firstweekstreamsandrank = findthePosonWeeksonChartperSong(za_data)
maxartistappearances = findtheMaxArtistAppearanceTimes(za_data)
fingroupeddf = groupeddf.merge(maxsongperartistdf, left_on=["main_artist"], right_on=["main_artist"], how="inner")
fingroupeddf = fingroupeddf.drop(["Artist Song Count"],axis=1)
fingroupeddf = fingroupeddf.merge(weeksonchartpersong, left_on=groupby_cols, right_on=groupby_cols, how="inner")
fingroupeddf = fingroupeddf.merge(firstweekstreamsandrank, left_on=groupby_cols, right_on=groupby_cols, how="inner")
fingroupeddf = fingroupeddf.merge(maxartistappearances, left_on=["main_artist"], right_on=["main_artist"], how="inner")

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(za_data["source"])

za_encoded_source = le.transform(za_data["source"])
za_data["encoded_source"] = za_encoded_source
zarelevantsource = za_data[["main_artist", "track_name", "encoded_source"]]
zarelevantsource = zarelevantsource.drop_duplicates()

fingroupeddf = fingroupeddf.merge(zarelevantsource, left_on=groupby_cols, right_on=groupby_cols, how="inner")
fingroupeddf.to_csv("Classification/cleanedartistsongglobal.csv", index=False)

In [35]:
maxsongperartistdftest = findtheMinMaxNumofSongs(groupeddftest)
weeksonchartpersongtest = findtheWeeksonChartperSong(za_data_test_first_time)
firstweekstreamsandranktest = findthePosonWeeksonChartperSong(za_data_test_first_time)
maxartistappearancestest = findtheMaxArtistAppearanceTimes(za_data_test_first_time)
fingroupeddftest = groupeddftest.merge(maxsongperartistdftest, left_on=["main_artist"], right_on=["main_artist"], how="inner")
fingroupeddftest = fingroupeddftest.drop(["Artist Song Count"],axis=1)
fingroupeddftest = fingroupeddftest.merge(weeksonchartpersongtest, left_on=groupby_cols, right_on=groupby_cols, how="inner")
fingroupeddftest = fingroupeddftest.merge(firstweekstreamsandranktest, left_on=groupby_cols, right_on=groupby_cols, how="inner")
fingroupeddftest = fingroupeddftest.merge(maxartistappearancestest, left_on=["main_artist"], right_on=["main_artist"], how="inner")

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(za_data_test_first_time["source"])

za_encoded_sourcetest = le.transform(za_data_test_first_time["source"])
za_data_test_first_time["encoded_source"] = za_encoded_sourcetest
zarelevantsourcetest = za_data_test_first_time[["main_artist", "track_name", "encoded_source"]]
zarelevantsourcetest = zarelevantsourcetest.drop_duplicates()

fingroupeddftest = fingroupeddftest.merge(zarelevantsourcetest, left_on=groupby_cols, right_on=groupby_cols, how="inner")
fingroupeddftest.to_csv("Classification/cleanedartistsongglobaltest.csv", index=False)

In [36]:
fingroupeddftest

,main_artist,track_name,Max Streams per Song,Min Streams per Song,Total Streams per Song,Mean Streams per Song,Min Position per Song,MaxMin Difference Position,MaxMin Difference Streams,MaxMin Ratio Position,MaxMin Ratio Streams,Max Song Count per artist,Max Weeks on Chart per song,first_week_rank,first_week_streams,Max Artist Appearance,encoded_source
0,$,BACK TO ME,19028110,19028110,19028110,19028110.0,24,0,0,1.0,1.0,1,1,24,19028110,48,370
1,$,BEG FORGIVENESS,11802992,11802992,11802992,11802992.0,89,0,0,1.0,1.0,1,1,89,11802992,48,370
2,$,BURN,19646355,19646355,19646355,19646355.0,21,0,0,1.0,1.0,1,1,21,19646355,48,370
3,$,CARNIVAL,32277135,32277135,32277135,32277135.0,4,0,0,1.0,1.0,1,1,4,32277135,48,370
4,$,DO IT,14290306,14290306,14290306,14290306.0,59,0,0,1.0,1.0,1,1,59,14290306,48,370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288,juan karlos,ERE,10257602,10257602,10257602,10257602.0,101,0,0,1.0,1.0,1,1,101,10257602,0,329
1289,sombr,back to friends,11666265,11666265,11666265,11666265.0,83,0,0,1.0,1.0,1,1,83,11666265,2,354
1290,sombr,undressed,10016074,10016074,10016074,10016074.0,123,0,0,1.0,1.0,1,1,123,10016074,2,354
1291,wifiskeleton,Nope your too late i already died,10352667,10352667,10352667,10352667.0,111,0,0,1.0,1.0,1,1,111,10352667,0,16


In [37]:
correlationX = X.corr()

NameError: name 'X' is not defined

In [ ]:
thecols = list(correlationX.columns)

In [ ]:
allvals = {}
for acol in thecols:
    for innercol in thecols:
        if(correlationX[acol][innercol] > 0.7 and correlationX[acol][innercol] < 1):
           allvals[acol + "+" + innercol] = correlationX[acol][innercol]
        

In [ ]:
dropcolvalsall = []
for newval in list(allvals.keys()):
    dropcolvals = newval.split("+")[1]
    dropcolvalsall.append(dropcolvals)

In [ ]:
thedropcolsvf = list(set(dropcolvalsall))

In [ ]:
thecols

In [ ]:
# correlationX

##### LINEAR REGRESSION NUMBER OF WEEKS

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
fingroupeddf = fingroupeddf.reset_index()
fingroupeddf = fingroupeddf.drop(["index"], axis=1)

In [ ]:
fingroupeddf.shape

In [ ]:
fingroupeddf.columns

In [ ]:
# X = fingroupeddf.drop(["main_artist", "track_name", 'Max Weeks on Chart per song'] + thedropcolsvf, axis=1)
X = fingroupeddf.drop(["main_artist", "track_name", "Max Weeks on Chart per song", 'Max Artist Appearance'], axis=1)
# X = X[['first_week_rank', 'first_week_streams','Total Streams per Song', 'Min Position per Song', 'MaxMin Difference Position', 'MaxMin Ratio Position', 'MaxMin Ratio Streams']]
# X = X[['MaxMin Difference Position', 'MaxMin Ratio Streams', "first_week_streams"]]
y = fingroupeddf["Max Weeks on Chart per song"]

In [ ]:
thecols = list(X.columns)

In [ ]:
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [ ]:
X = pd.DataFrame(X)

In [ ]:
X.columns = thecols

In [ ]:
thecols

In [ ]:
X_train = X.head(4000)
y_train = y.head(4000)

X_test = X.iloc[4001:]
y_test = y.iloc[4001:]

# X_train = X.head(1800)
# y_train = y.head(1800)

# X_test = X.iloc[1801:]
# y_test = y.iloc[1801:]

In [ ]:
reg = LinearRegression().fit(X_train, y_train)
print(reg.score(X, y))
print(reg.coef_)
print(reg.intercept_)

In [ ]:
all_preds = reg.predict(X_test)
thepredsy = pd.DataFrame(all_preds)

thepredsy.columns = ["predictions"]
thepredsy["actuals"] = list(y_test)
thepredsy["predictions"] = thepredsy["predictions"].astype(int)
thepredsy["diff"] = thepredsy["predictions"] - thepredsy["actuals"]
thepredsy.to_csv("Classification/thepredsy.csv", index=False)

truepreds = thepredsy[thepredsy["diff"]==0]
truepreds1 = thepredsy[thepredsy["diff"]==1]
truepreds2 = thepredsy[thepredsy["diff"]==-1]
fintruepeds = pd.concat([truepreds,truepreds1,truepreds2])

print("Coefficient of determination: %.2f" % r2_score(y_test, all_preds))

In [ ]:
mae = mean_absolute_error(y_test, all_preds)

In [ ]:
mae

In [ ]:
truepreds.shape[0], fintruepeds.shape[0], all_preds.shape[0]

In [ ]:
(truepreds.shape[0] / thepredsy.shape[0]) * 100, (fintruepeds.shape[0] / thepredsy.shape[0]) * 100

In [ ]:
thediffmore = thepredsy.groupby("diff")["predictions"].count().reset_index()

In [ ]:
# thediffmore.to_csv("Classification/diffmore.csv")

In [ ]:
# X[["first_week_rank", "first_week_streams"]] - 7 , 1.26%
# ["first_week_rank", "first_week_streams", "Max Streams per Song"] - 15 , 2.69%
# [all] - 48 , 8.62%
# X[["first_week_rank", "first_week_streams", "Max Streams per Song", "Total Streams per Song"]] - 28 , 5.02%

# X[["first_week_rank", "first_week_streams", "Max Streams per Song", "Total Streams per Song", 'Max Song Count per artist']] - 37, 6.64% 

# X[["Mean Streams per Song", "first_week_rank"]]- 8 , 1.44%

# ["MaxMin Ratio Position", "first_week_rank", "Max Song Count per artist" - 7 , 1.26%

In [ ]:
from sklearn import linear_model
reglinear_model = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))
reglinear_modelvf = reglinear_model.fit(X_train, y_train)

In [ ]:
all_preds = reglinear_modelvf.predict(X_test)
thepredsy = pd.DataFrame(all_preds)

thepredsy.columns = ["predictions"]
thepredsy["actuals"] = list(y_test)
thepredsy["predictions"] = thepredsy["predictions"].astype(int)
thepredsy["diff"] = thepredsy["predictions"] - thepredsy["actuals"]

truepreds = thepredsy[thepredsy["diff"]==0]
truepreds1 = thepredsy[thepredsy["diff"]==1]
truepreds2 = thepredsy[thepredsy["diff"]==-1]
fintruepeds = pd.concat([truepreds,truepreds1,truepreds2])

In [ ]:
print("Coefficient of determination: %.2f" % r2_score(y_test, all_preds))

In [ ]:
mae = mean_absolute_error(y_test, all_preds)
mae

In [ ]:
truepreds.shape[0], fintruepeds.shape[0]

In [ ]:
(truepreds.shape[0] / thepredsy.shape[0]) * 100, (fintruepeds.shape[0] / thepredsy.shape[0]) * 100

In [ ]:
bayesianreg = linear_model.BayesianRidge()
bayesianregvf = bayesianreg.fit(X_train, y_train)

all_preds = bayesianregvf.predict(X_test)
thepredsy = pd.DataFrame(all_preds)

thepredsy.columns = ["predictions"]
thepredsy["actuals"] = list(y_test)
thepredsy["predictions"] = thepredsy["predictions"].astype(int)
thepredsy["diff"] = thepredsy["predictions"] - thepredsy["actuals"]

truepreds = thepredsy[thepredsy["diff"]==0]
truepreds1 = thepredsy[thepredsy["diff"]==1]
truepreds2 = thepredsy[thepredsy["diff"]==-1]
fintruepeds = pd.concat([truepreds,truepreds1,truepreds2])

truepreds.shape[0], fintruepeds.shape[0]

In [ ]:
print("Coefficient of determination: %.2f" % r2_score(y_test, all_preds))

In [ ]:
mae = mean_absolute_error(y_test, all_preds)
mae

In [ ]:
(truepreds.shape[0] / thepredsy.shape[0]) * 100, (fintruepeds.shape[0] / thepredsy.shape[0]) * 100

In [ ]:
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot

importance = reg.coef_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()